# Yolov3 Dataloader (Local Machine)

![](https://img.shields.io/static/v1?label=Python&message=3.6&color=green)
![](https://img.shields.io/static/v1?label=ONNX&message=1.4.1&color=yellow)
![](https://img.shields.io/static/v1?label=TRT&message=6.0.1&color=orange)
![](https://img.shields.io/static/v1?label=License&message=MIT&color=red)

*** Author: Kevin Yu (www.github.com/yqlbu)

*** Email: kevinyu211@yahoo.com

- This tool is tailored for those who want to train their custom dataset on a Yolov3 Model. 

- Make sure you read the instructions for each step carefully !!!

- If you following the instructions below step by step, it will generate a new trained-weight in the end, and you may download it to your local machine. 

- You might need to **enable scrolling for cell outputs** by default. To do so, highlight all of the cells (Cmd A or Ctrl A) and change them to **"scrolled"**.

- Make sure you open Jupyter Notebook or JupyterLab at **/TRT-yolov3** directory.

#### Good Luck!

# Setup

### Check your current path

In [ ]:
!pwd

**Notes:** If your path is **/home/username/TRT-yolov3/train**, you should be good to go. Otherwise, set path by executing the cell below.

In [ ]:
from os.path import expanduser
HOME = expanduser("~")
PATH = HOME+'/TRT-yolov3/train'
print(PATH)

### Install Matplotlib

In [ ]:
!pip3 install matplotlib --user

### Compile YOLOv3-Darknet with CUDA

In [ ]:
!sh train/install_darknet.sh

### Upload dataset

Create directories to store data files

In [ ]:
!mkdir data/
!pwd

Please follow the steps below to upload your dataset to the data folder located at **$HOME/TRT-yolov3/train/data**

#### Step #0: Prepare Dataset

**Notes:**

You may Use the [ImgLabel](https://github.com/tzutalin/labelImg) tool to do the labeling work.

- The dataset should contain all the images(.jpeg, .jpg, .png) and labels(.xml)
- Copy and paste all the image files and .xml files into **$HOME/TRT-yolov3/train/data**, or simply upload data with Jupyter's upload tool
- Rename data with **rename_data.py**
- Convert labels to txt format with **dataPatch.py**

#### Step #1: Rename Images

In [ ]:
%run rename_data.py

After the above operations, your **train/data/** directory should be similar as shown below:

```bash
├── data
│   ├── 00000.jpeg
│   ├── 00000.xml
│   ├── 00001.jpg
│   ├── 00001.xml
│   ├── 00002.jpeg
│   ├── 00002.xml
│   ├── 00003.jpg
│   ├── 00003.xml
│   ├── 00004.jpeg
│   └── 00004.xml
```

**Notes:** each image file should have a **txt** file and a **xml** file along with it.

*** Now you should be good to move forward ! Good Luck !

# Create config files for training

**Notes:** (Please read carefully before you excute the cells below):

 - Please make sure you set config parameters below corrently. Otherwise, you will not be able to activate the training process.
 - **MODEL_NAME** is the name of your new model
 - **CLASS_NAME** is associated with all the classes that your dataset contains, please fill in the CLASS_NAME list correctly with your classes
 - **CLASS_NUM** is associated with the total number of classes that your dataset contains
 - **MAX_BATCHES** is the total number of iterations in the training
 - **BATCH** is the number of data that set to feed for a single batch. Update parameters every N batches.
 - **SUBDIVISIONS**: Subdivisions serves as the number of groups. **BATCH/SUBDIVISIONS** is the number of data that set to feed into the training engine. 
 - **BATCH/SUBDIVISIONS**: if you machine does not have plenty of memory (>4G), it's better you set the **BATCH** < 64, bigger batch size would result in better training model; bigger subdivision may massively alleviate GPU pressure. since more groups means less sample required per group, the pressure that the GPU encounters would be much lower as a result.
 - If you set all the parameters mentioned above properly, you should be good to move forward

#### Step #0: Set Training Config Parameters

In [ ]:
import os
import getpass

#New Model Parameters

#Replace NEW_MODEL_NAME with your new model name
MODEL_NAME = 'NAME'
#Put your classes tag below
CLASS_NAME = [
    "Class_1",
    "Class_2"
]
#Replace # with your classes number (int), 2 is set as the default value
CLASS_NUM = 2
# Replace your desired batch # (6000 as default)
# Ideally the MAX_BATCHES = 2500*CLASS_NUM
MAX_BATCHES = 5000
# Ideally the BATCH_SIZE should be less than 64 if your memory is less than 8G
BATCH = 32
# SUBDIVISION is the # of groups
SUBDIVISIONS = 4
 
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #

USER=getpass.getuser()
PWD=os.path.abspath(os.getcwd())

#bash
%cd $PATH

#### Step #1: Convert Label to txt format

In [ ]:
#dataDispatch.py
with open('dataDispatch.py', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open('dataDispatch.py', "w") as file:
    all_lines[8] = 'classes = ' + str(CLASS_NAME) + '\n'
    print(all_lines[8].strip())
    file.writelines(all_lines)
    print('dataDispatch.py'+' has been successfully modified!')
    file.close()

In [ ]:
# run the convert script
%run dataDispatch.py

#### Step #2: Modify class.names

In [ ]:
#class.names
with open(MODEL_NAME+'.names', "a+") as file:
    file.seek(0)
    for name in CLASS_NAME:
        file.write(name + '\n')
    print(MODEL_NAME+'.names'+' has been successfully created!')
    file.close()

#### Step #3: Modify class.data

In [ ]:
#class.data
with open(MODEL_NAME+'.data', "a+") as file:
    file.seek(0)
    file.write('classes=' + str(CLASS_NUM) + '\n')
    file.write('train=' + PATH + '/train.txt' + '\n')
    file.write('valid=' + PATH + '/test.txt' + '\n')
    file.write('backup=' + PATH + '/backup' + '\n')
    file.write('names=' + PATH + '/'+ MODEL_NAME + '.names' + '\n')
    print(MODEL_NAME+'.data'+' has been successfully created!')
    file.close()

#### Step #4: Modify yolov3.cfg

**Notes:** By default, it is set to train a YOLOv3-tiny Model.

In [ ]:
#class-yolov3-tiny.cfg
with open('yolov3-tiny-config.txt', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open(MODEL_NAME+'-yolov3-tiny.cfg', "w") as file:
    #modify batches and subdivisions
    all_lines[5] = 'batch=' + str(BATCH) + '\n'
    all_lines[6] = 'subdivisions=' + str(SUBDIVISIONS) + '\n'
    #modify class #
    all_lines[134] = 'classes=' + str(CLASS_NUM) + '\n'
    all_lines[176] = 'classes=' + str(CLASS_NUM) + '\n'
    #modify max-batches
    all_lines[19] = 'max_batches = ' + str(MAX_BATCHES) + '\n'    
    #modify filter ## filter = (CLASS_NUM+5) *3 ##
    all_lines[126] = 'filters=' + str((CLASS_NUM+5) *3) + '\n'
    all_lines[170] = 'filters=' + str((CLASS_NUM+5) *3) + '\n'
    #overwrite all
    file.writelines(all_lines)
    print(MODEL_NAME+'-yolov3-tiny.cfg'+' has been successfully created!')
    file.close()

#### Step #5: Modify train.sh

In [ ]:
#train.sh
with open('train.sh', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open('train.sh', "w") as file:
    all_lines[0] = 'export DARKNET=' + PATH + '/darknet' + '\n'
    all_lines[1] = 'export PRJ_PATH=' + PATH + '\n'
    all_lines[7] = '$PRJ_PATH/' + MODEL_NAME + '.data \\' + '\n'
    all_lines[8] = '$PRJ_PATH/' + MODEL_NAME +'-yolov3-tiny.cfg' + ' \\' + '\n'
    file.writelines(all_lines)
    print('train.sh'+' has been successfully modified!')
    file.close()

*** Now you should be good to move forward ! Good Luck !

# Training

**Notes:**

 - Navigate to the **$HOME/TRT-yolov3/train/** directory
 - Check if the directory contains the **".data"**, the **".names"**, and the **".cfg"** files. If you miss one or more of the files, please check the instructions from the above steps.
 - You might need to **enable scrolling for cell outputs** to fix the cell output window. You may find the option by **right clicking** the cell below.
 - Now you should be good to activate the training process, good luck !

In [ ]:
#start training
!mkdir backup
!sudo chmod +x $PWD/darknet/darknet
!sudo sh train.sh
!echo '[BASH]  Congratulations! Training has completed. Enjoy!'

**Notes:** Once the training has finished, the final weight is stored in the **$HOME/TRT-yolov3/train/backup** directory.

Your**/backup** directory should look something similar as shown below:

```bash
├── mask-yolov3-tiny_1000.weights
├── mask-yolov3-tiny_2000.weights
├── mask-yolov3-tiny_3000.weights
├── mask-yolov3-tiny_4000.weights
├── mask-yolov3-tiny_5000.weights
├── mask-yolov3-tiny_final.weights
└── mask-yolov3-tiny_last.weights
```

**Constantly monitor the training progress**

- Option #1: You may observe a chart of how your model did after the training process by running the below command. It shows a chart of your average loss vs. iterations.

- Option #2: You may open the **chart.png** located in **TRT-yolov3/train/darknet/** with Jupyter anytime during the training process.

- Option #3: You may open **show_progress.ipynb** notebook to constantly observe the progress.

- For your model to be **'accurate'** you would aim for a loss under 1.

In [ ]:
#Show Training process
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

IMG_PATH = PATH + '/darknet/chart.png'
imShow(IMG_PATH)

# Test

#### Step #1: Download an Image from the Internet to test your model

In [ ]:
!mkdir img

In [ ]:
def ImgDownload(img_url,img_name):
  import urllib.request
  print('Image downloading...')
  img_path = PATH + '/img/'
  img_name = img_name + '.jpg'
  urllib.request.urlretrieve(img_url, img_path+img_name)
  print('Image displaying...')
  imShow(img_path+img_name)

img_url = 'IMG_URL'
# eg: img_url = 'https://tasteasianfood.com/wp-content/uploads/2016/01/et17-720x405.jpg'
img_name = 'IMG_NAME'
# eg: img_name = 'eggtart'
ImgDownload(img_url,img_name)

#### Step #2: Modify the demo script

In [ ]:
#demo.sh
with open('demo.sh', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open('demo.sh', "w") as file:
    all_lines[0] = 'export DARKNET=' + PATH + '/darknet' + '\n'
    all_lines[1] = 'export PRJ_PATH=' + PATH + '\n'
    all_lines[5] = '$PRJ_PATH/' + MODEL_NAME + '.data \\' + '\n'
    all_lines[6] = '$PRJ_PATH/' + MODEL_NAME+'-yolov3-tiny.cfg' + ' \\' + '\n'
    all_lines[7] = '$PRJ_PATH/backup/' + MODEL_NAME+'-yolov3-tiny_final.weights' + ' \\' + '\n'
    all_lines[8] = '-ext_output ' + '$PRJ_PATH/img/' + img_name + '.jpg' + ' \\' + '\n'
    file.writelines(all_lines)
    print('demo.sh'+' has been successfully modified!')
    print('Ready to predict ' + img_name + '.jpg')
    file.close()

#### Step #3: Run the Demo Script

In [ ]:
!sudo chmod +x $PWD/darknet/darknet
!sh demo.sh

#### Step #4: Display the prediction

In [ ]:
imShow(PATH + '/darknet/predictions.jpg')

# Next

Now that you have tested the new trained weights, let's move on to convert YOLOv3 Weight to TensorRT !

**Notes:** Please read README.md carefully for further instructions